In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
"""
This Notebook will be tha base for all the YOLO training that Reetta does in the project.
"""

In [ ]:
if torch.cuda.is_available():
    device = 0
    print(f"Training on GPU")
else:
    print(f"Training on CPU")
    device = "cpu"

In [ ]:
"""
Default values used for all YOLO trainings.

These parameters should remain consistent across different training runs,
unless the hardware or training context changes significantly.
"""

model = YOLO("yolo11l.pt")  # pretrained YOLOv11L model

default_args = {
    "data": "config/materials.yaml",     # dataset configuration path
    "imgsz": 640,                     # default input size for YOLO
    "batch": 16,                      # maximum batch size (GPU memory limit)
    "device": 0,                      # GPU device ID
    "cache": "ram",                   # cache data in RAM for speed
    "workers": 4,                     # parallel data loader workers
    "amp": True,                      # automatic mixed precision (faster training)
    "close_mosaic": 10,               # disable mosaic in last N epochs
    "save": True,                     # save checkpoints and weights
    "val": True,                      # enable validation after each epoch
    "plots": True                     # save training/validation plots
}

In [ ]:
"""
Run-specific training parameters.

These parameters can be adjusted per experiment to test different configurations,
optimizers, learning rates, or other hyperparameters.
"""

results = model.train(
    **default_args,
    epochs=50,                        # number of training epochs
    optimizer="AdamW",                # optimizer: AdamW, Adam, or SGD
    lr0=0.001,                        # initial learning rate
    momentum=0.85,                     # used with optimizers like SGD or AdamW
    weight_decay=0.0002,              # regularization to prevent overfitting
    warmup_epochs=5.0,                # warm-up period for learning rate
    project="material_koulutus",         # output base folder
    name="train_reetta_v8"             # unique name for this training run
)

In [ ]:
"""
This script reads YOLO training results from a CSV file and visualizes key evaluation metrics:
- Training and validation losses
- Precision and recall over epochs
- mAP scores at IoU 0.5 and 0.5–0.95
- F1 score per epoch (calculated)
- Confusion matrix (final)
"""

%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

csv_path = "material_koulutus/train_reetta_v4/results.csv"
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()

display(df.head())
display(df.tail())

df[["train/box_loss", "train/cls_loss", "train/dfl_loss"]].plot(figsize=(10, 5), title="Training Losses per Epoch")
plt.xlabel("Epoch")
plt.grid()
plt.show()

df[["val/box_loss", "val/cls_loss", "val/dfl_loss"]].plot(figsize=(10, 5), title="Validation Losses per Epoch")
plt.xlabel("Epoch")
plt.grid()
plt.show()

df[["metrics/precision(B)", "metrics/recall(B)"]].plot(figsize=(10, 5), title="Precision and Recall")
plt.xlabel("Epoch")
plt.grid()
plt.show()

df[["metrics/mAP50(B)", "metrics/mAP50-95(B)"]].plot(figsize=(10, 5), title="mAP Scores")
plt.xlabel("Epoch")
plt.grid()
plt.show()

df[["lr/pg0", "lr/pg1", "lr/pg2"]].plot(figsize=(10, 5), title="Learning Rates")
plt.xlabel("Epoch")
plt.grid()
plt.show()

if "metrics/precision(B)" in df.columns and "metrics/recall(B)" in df.columns:
    precision = df["metrics/precision(B)"]
    recall = df["metrics/recall(B)"]
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    plt.figure(figsize=(10, 5))
    plt.plot(df["epoch"], f1, label="F1 Score")
    plt.title("F1 Score per Epoch")
    plt.xlabel("Epoch")
    plt.grid()
    plt.legend()
    plt.show()

cm_path = "material_koulutus/train_reetta_v8/confusion_matrix.png"
if os.path.exists(cm_path):
    img = Image.open(cm_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title("Confusion Matrix (Final Epoch)")
    plt.show()